<a href="https://colab.research.google.com/github/Material-Educativo/Tecnicas-heuristicas/blob/main/Evolucion_diferencial_vs_Alpine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Función Alpine
Empecemos viendo la regla de correspondencia y gráfica de la función Alpine.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
def alpine(x, y):
    """Calcula el valor de la funcion Alpine."""
    return np.abs(x * np.sin(x) + 0.1 * x) + np.abs(y * np.sin(y) + 0.1 * y)

# Evolución diferencial (ED)
Veamos el comportamiento de evolución diferencial con la función alpine.

In [ ]:
def evolucion_diferencial(max_iter, tam_poblacion, F, CR,
                          rango_x=(-5, 5), rango_y=(-5, 5)):
    """Optimiza la funcion Alpine usando evolucion diferencial."""
    # Inicializar poblacion con individuos aleatorios
    poblacion = [
        (random.uniform(rango_x[0], rango_x[1]),
         random.uniform(rango_y[0], rango_y[1]))
        for _ in range(tam_poblacion)
    ]

    # Evaluar funcion objetivo para cada individuo
    costos = [alpine(x, y) for x, y in poblacion]

    # Identificar mejor individuo de la poblacion inicial
    indice_mejor = np.argmin(costos)
    mejor_individuo = poblacion[indice_mejor]
    mejor_valor = costos[indice_mejor]

    # Preparar historial para analisis
    historial_valores = [mejor_valor]
    coordenadas_x = [np.array([ind[0] for ind in poblacion])]
    coordenadas_y = [np.array([ind[1] for ind in poblacion])]

    for generacion in range(max_iter):
        for i in range(tam_poblacion):
            # Seleccionar tres individuos aleatorios diferentes
            # r1, r2, r3 deben ser distintos entre si y distintos de i
            indices_disponibles = [j for j in range(tam_poblacion) if j != i]
            r1, r2, r3 = random.sample(indices_disponibles, 3)

            # MUTACION: Generar vector donante (mutante)
            vector_base = np.array(poblacion[r1])
            diferencia = np.array(poblacion[r2]) - np.array(poblacion[r3])
            vector_donante = vector_base + F * diferencia

            # CRUZA BINOMIAL: Combinar vector objetivo con vector donante
            # Seleccionar al menos una coordenada del donante
            coor_obligatoria = random.randint(0, 1)  # 0 para x, 1 para y

            # Crear vector de prueba (hijo)
            vector_prueba = []

            for j in range(2):  # Iterar sobre las dos dimensiones (x, y)
                # Heredar del donante si: cumple CR O es la coordenada obligatoria
                if random.random() < CR or j == coor_obligatoria:
                    vector_prueba.append(vector_donante[j])
                else:
                    # Heredar del objetivo
                    vector_prueba.append(poblacion[i][j])

            # Extraer coordenadas del vector de prueba
            x_prueba, y_prueba = vector_prueba[0], vector_prueba[1]

            # Asegurar que el vector de prueba este dentro de los limites
            x_prueba = np.clip(x_prueba, rango_x[0], rango_x[1])
            y_prueba = np.clip(y_prueba, rango_y[0], rango_y[1])

            # Evaluar funcion objetivo del vector de prueba
            valor_prueba = alpine(x_prueba, y_prueba)

            # SELECCION: Reemplazar si el hijo es igual o mejor que el padre
            if valor_prueba <= costos[i]:
                poblacion[i] = (x_prueba, y_prueba)
                costos[i] = valor_prueba

        # Actualizar mejor solucion de la generacion
        indice_mejor = np.argmin(costos)
        mejor_individuo = poblacion[indice_mejor]
        mejor_valor = costos[indice_mejor]

        # Guardar historial
        historial_valores.append(mejor_valor)
        coordenadas_x.append(np.array([ind[0] for ind in poblacion]))
        coordenadas_y.append(np.array([ind[1] for ind in poblacion]))

        # Retornar mejor solucion encontrada y su historial
    return (mejor_individuo[0], mejor_individuo[1], mejor_valor,
            historial_valores, coordenadas_x, coordenadas_y)

## Configuración de parámetros y ejecución del algoritmo.

In [ ]:
# === Configuracion de parametros ===
max_iter = 20
tam_poblacion = 10
F = 0.5   # Factor de mutacion
CR = 0.7  # Tasa de cruza

# === Ejecutar evolucion diferencial ===
x_opt, y_opt, valor_opt, historial, coord_x, coord_y = evolucion_diferencial(
    max_iter=max_iter,
    tam_poblacion=tam_poblacion,
    F=F,
    CR=CR
)

# === Imprimir resultados ===
print("Mejor solucion encontrada por DE:")
print(f"  X = {x_opt:.6f}")
print(f"  Y = {y_opt:.6f}")
print(f"  Valor de Alpine = {valor_opt:.6f}")
print(f"  Optimo teorico: f(0, 0) = 0")

# Veamos una gráfica de convergencia

In [ ]:
# === Grafica de convergencia ===
plt.figure(figsize=(10, 6))
generaciones = range(len(historial))

plt.plot(generaciones, historial, 'b-', linewidth=2, label='Mejor valor')
plt.scatter(generaciones, historial, c='red', s=30, zorder=5)

plt.xlabel('Generacion', fontsize=12)
plt.ylabel('Mejor valor encontrado', fontsize=12)
plt.title('Convergencia de evolucion diferencial en funcion Alpine',
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Veamos la trayectoria de un individuo

In [ ]:
# === Crear malla para curvas de nivel ===
x_vals = np.linspace(-5.0, 5.0, 100)
y_vals = np.linspace(-5.0, 5.0, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = alpine(X, Y)

# === Configurar figura ===
plt.figure(figsize=(10, 8))

# Graficar curvas de nivel
plt.contour(X, Y, Z, levels=20, colors='gray', alpha=0.5)
plt.contourf(X, Y, Z, levels=30, cmap='viridis', alpha=0.6)

# === Seleccionar individuo a visualizar ===
individuo_id = 2

# === Graficar trayectoria ===
plt.plot(
    coord_x[individuo_id],
    coord_y[individuo_id],
    color='red',
    marker='o',
    markersize=6,
    linestyle='-',
    linewidth=2,
    label=f'Individuo {individuo_id}'
)

# === Agregar flechas direccionales ===
for i in range(1, len(coord_x[individuo_id]) - 1):
    dx = (coord_x[individuo_id][i] - coord_x[individuo_id][i-1]) * 0.5
    dy = (coord_y[individuo_id][i] - coord_y[individuo_id][i-1]) * 0.5

    plt.arrow(
        coord_x[individuo_id][i-1],
        coord_y[individuo_id][i-1],
        dx, dy,
        color='black',
        head_width=0.1,
        head_length=0.1,
        zorder=10
    )

plt.colorbar(label='Valor de Alpine')
plt.xlabel('X', fontsize=12)
plt.ylabel('Y', fontsize=12)
plt.title(f'Trayectoria del individuo {individuo_id} en DE',
          fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

#Veamos una animación

In [ ]:
import matplotlib.animation as animation
from IPython.display import Video  # Para visualizar en Colab

In [ ]:
# === Crear malla para curvas de nivel ===
x_vals = np.linspace(-5.0, 5.0, 100)
y_vals = np.linspace(-5.0, 5.0, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = alpine(X, Y)

# === Configurar figura y ejes ===
fig, ax = plt.subplots(figsize=(8, 7))

# Graficar curvas de nivel
contour = ax.contourf(X, Y, Z, levels=30, cmap='viridis')
ax.contour(X, Y, Z, levels=20, colors='black', alpha=0.3)

# Agregar barra de color
plt.colorbar(contour, ax=ax, label='Valor de Alpine')

# Configurar limites y etiquetas
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_xlabel('X', fontsize=12)
ax.set_ylabel('Y', fontsize=12)
ax.set_title('Evolucion de la poblacion en DE', fontsize=14, fontweight='bold')

# === Inicializar circulos para cada individuo ===
circles = [
    plt.Circle((coord_x[0][i], coord_y[0][i]), 0.15,
               color='blue', zorder=5)
    for i in range(tam_poblacion)
]

# Agregar circulos al grafico
for circle in circles:
    ax.add_artist(circle)

# === Funcion de actualizacion para cada frame ===
def actualizar_frame(frame):
    """Actualiza la posicion de cada individuo en la generacion dada."""
    for i, circle in enumerate(circles):
        # Actualizar centro del circulo a la nueva posicion
        nueva_posicion = (coord_x[frame][i], coord_y[frame][i])
        circle.set_center(nueva_posicion)

    return circles
# === Crear animacion ===
animacion = animation.FuncAnimation(
    fig,
    actualizar_frame,
    frames=len(coord_x),
    interval=500,  # 500 ms entre frames (medio segundo)
    blit=True,
    repeat=True
)

# === Guardar como archivo MP4 ===
animacion.save(
    'ed_animacion.mp4',
    writer='ffmpeg',
    fps=2,  # 2 frames por segundo
    dpi=100
)

print("Animacion guardada como: ed_animacion.mp4")

# === Visualizar en Google Colab ===
Video('ed_animacion.mp4', embed=True)